In [29]:
# clear all variables from the notebook
for name in dir():
    if not name.startswith('_'):
        del globals()[name]

import sympy as sp
import numpy as np

Ts_val = 5.3            # [s]               (Tiempo de asentamiento)
os = 0.13               # [%/100]           (Overshoot)


# Define the symbols
s = sp.symbols('s', complex=True)
Ts, xi, wn = sp.symbols('T_S \\xi \\omega_n', positive=True, real=True)
k1, k2, ke = sp.symbols('k_1 k_2 k_e')

xi_val = sp.simplify((-sp.ln(os) / sp.sqrt(sp.pi**2 + sp.ln(os)**2))).evalf()
wn_val = sp.simplify(4 / (xi_val * Ts_val)).evalf()
S_roots = [(-xi_val*wn_val + j*wn_val*sp.sqrt(1 - xi_val**2)).evalf() for j in [sp.I, -sp.I]]

display(sp.Eq(xi, sp.N(xi_val, 5)), sp.Eq(wn, sp.N(wn_val, 5)))
print(f"complex poles |S_i| = { np.abs(complex(sp.N(S_roots[0]))):.03f}")
for i, root in enumerate(S_roots):
    root = sp.N(root, 5)
    # print(f"S_{i+1}: {str(root).ljust(20)} = {np.abs(root)}  /_  {np.angle(complex(root)):0.2f} rad")
    display(sp.Eq(sp.symbols(f'S_{i+1}'), root))
print(f"S_roots_planta = {S_roots}")

Eq(\xi, 0.54465)

Eq(\omega_n, 1.3857)

complex poles |S_i| = 1.386


Eq(S_1, -0.75472 + 1.1621*I)

Eq(S_2, -0.75472 - 1.1621*I)

S_roots_planta = [-0.754716981132076 + 1.16213563272773*I, -0.754716981132076 - 1.16213563272773*I]


In [14]:
# Agregar el 3er Polo
S_3 = -100              # [rad/s]           (3er polo)

# Define A, B, C matrices
A = [   [0,     1       ], 
        [0,     -261.72 ]]

B = [   [0      ], 
        [244.68 ]]

C = [   [1,     0]  ]


# Ya definieron todo lo necesario para calcular, ahora es solo ver los resultados
A = sp.Matrix(A)
B = sp.Matrix(B)
C = sp.Matrix(C)

if len(S_roots) > 2:
    S_roots.pop(2)
S_roots.append(S_3)

print("Response parameters:")

desired_poly = 1
for root in S_roots:
    desired_poly *= (s - root)
desired_poly = sp.N(desired_poly, 5)

print("Desired polynomial:")
display(sp.Eq(sp.symbols('P_d'), desired_poly))
desired_poly = desired_poly.as_poly(s).as_expr()
display(sp.Eq(sp.symbols('P_d'), desired_poly, evaluate=False))


Aa = sp.MatrixSymbol('A_a', 3, 3)
Ba = sp.MatrixSymbol('B_a', 3, 1)
Ca = sp.MatrixSymbol('C_a', 3, 3)

Aa_val = sp.Matrix([[A, sp.zeros(2, 1)], [-C, sp.zeros(1, 1)]])
Ba_val = sp.Matrix([B, sp.zeros(1, 1)])
Ca_val = sp.Matrix([[C, sp.zeros(1, 1)]])
k = sp.Matrix([[k1, k2, -ke]])

display(sp.Eq(Aa, Aa_val, evaluate=False), sp.Eq(Ba, Ba_val, evaluate=False), sp.Eq(Ca, Ca_val, evaluate=False))

pole_place_eq = sp.Eq( sp.Determinant(s * sp.Identity(3) - Aa + Ba*k), 0)
print("Pole placement equation:")
display(pole_place_eq)

pole_place_eq = pole_place_eq.subs(Aa, Aa_val).subs(Ba, Ba_val).subs(sp.Identity(3), sp.eye(3)).expand().doit()
print("with values")
display(pole_place_eq)
# combine the terms

pole_place_eq = sp.Eq(sp.Determinant(pole_place_eq.lhs.arg.doit()), 0)
print("combined")
display(pole_place_eq)

pole_place_eq = pole_place_eq.doit().as_poly(s).as_expr()
print("simplified")
display(sp.Eq(pole_place_eq, 0))

Response parameters:
Desired polynomial:


Eq(P_d, (s + 100.0)*(s + 0.75472 - 1.1621*I)*(s + 0.75472 + 1.1621*I))

Eq(P_d, 1.0*s**3 + 101.51*s**2 + 152.86*s + 192.02)

Eq(A_a, Matrix([
[ 0,       1, 0],
[ 0, -261.72, 0],
[-1,       0, 0]]))

Eq(B_a, Matrix([
[     0],
[244.68],
[     0]]))

...

Eq(Determinant(Matrix([
[0,     -1, 0],
[0, 261.72, 0],
[1,      0, 0]]) + Matrix([
[         0,          0,           0],
[244.68*k_1, 244.68*k_2, -244.68*k_e],
[         0,          0,           0]]) + Matrix([
[s, 0, 0],
[0, s, 0],
[0, 0, s]])), 0)

combined


Eq(Determinant(Matrix([
[         s,                      -1,           0],
[244.68*k_1, 244.68*k_2 + s + 261.72, -244.68*k_e],
[         1,                       0,           s]])), 0)

simplified


Eq(244.68*k_1*s + 244.68*k_e + 1.0*s**3 + s**2*(244.68*k_2 + 261.72), 0)

In [22]:
# iterate both polynomials to get the coefficients
eqs = []
for i in range(4):
    lhs = desired_poly.as_poly(s).coeffs()[i]
    rhs = pole_place_eq.as_poly(s).coeffs()[i]
    eqs.append(sp.Eq(lhs, rhs))

sol = sp.solve(eqs, (k1, k2, ke))

print("Solution:")
for key, val in sol.items():
    display(sp.Eq(key, sp.N(val, 4)))

k1_val, k2_val, ke_val = [val.evalf() for val in sol.values()]
display(sp.Eq(sp.Symbol('k'), k, evaluate=False))

Solution:


Eq(k_1, 0.6247)

Eq(k_2, -0.6548)

Eq(k_e, 0.7848)

Eq(k, Matrix([[k_1, k_2, -k_e]]))

# Transferencia a lazo cerrado

In [28]:
Tf = 1
for Si in S_roots:
    Tf *= 1/(s - Si)

Tf = sp.N(Tf.simplify(), 5)
Kg = 1/Tf.subs(s, 0)
Tf = Tf * Kg

display(sp.Eq(sp.symbols('T(s)'), Tf))

Eq(T(s), 192.02/(1.0*s**3 + 101.51*s**2 + 152.86*s + 192.02))

# Error permanente

$ e_{ss} = \lim_{s \to 0} s R(s) \left[ 1 - C \left( s I - A_{cl} \right)^{-1} \right] $

$ \left( s \to 0 \right)  \implies  e_{ss} = \left[ 1 - C \left(A_{cl} \right)^{-1} \right] $

In [9]:
# permanent error
Rs = sp.Function('R')(s)

# matrix to scalar value function
Acl = A - B * sp.Matrix([[k1_val, k2_val]])

ess = sp.Limit(s*Rs*(1 - (C*(Acl).inv())[0]), s, 0)

# ess = ess.subs(Rs, 1/s)
# ess.evalf()

ess

Limit(1.23612055291185*s*R(s), s, 0, dir='+')